In [27]:
from distutils.version import StrictVersion
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import time
import serial

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import time
import serial


# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')

In [28]:

# This is needed to display the images.
%matplotlib inline

In [29]:
from utils import label_map_util

from utils import visualization_utils as vis_util

#Variables

In [30]:

# What model to download.
MODEL_NAME = 'inference_graph'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('training', 'labelmap.pbtxt')

NUM_CLASSES = 6

In [31]:
# opener = urllib.request.URLopener()
# opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
# tar_file = tarfile.open(MODEL_FILE)
# for file in tar_file.getmembers():
#   file_name = os.path.basename(file.name)
#   if 'frozen_inference_graph.pb' in file_name:
#     tar_file.extract(file, os.getcwd())

In [32]:
##LOAD FROZEN TENSORFLOW GRAPH##
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [33]:
##Loading label map##
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


In [34]:
##Helper code loades it into numpy array##
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  if image.format == 'PNG':
      image = image.convert('RGB')
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [35]:
import pathlib


# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('C:/Users/test/models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpeg")))
TEST_IMAGE_PATHS

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [36]:
##DETECTION##

def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [37]:
# def draw_bounding_box_on_image(image,
#                            ymin,
#                            xmin,
#                            ymax,
#                            xmax,
#                            color='red',
#                            thickness=4,
#                            display_str_list=(),
#                            use_normalized_coordinates=True):
#   """Adds a bounding box to an image.
#   Bounding box coordinates can be specified in either absolute (pixel) or
#   normalized coordinates by setting the use_normalized_coordinates argument.
#   Each string in display_str_list is displayed on a separate line above the
#   bounding box in black text on a rectangle filled with the input 'color'.
#   If the top of the bounding box extends to the edge of the image, the strings
#   are displayed below the bounding box.
#   Args:
#     image: a PIL.Image object.
#     ymin: ymin of bounding box.
#     xmin: xmin of bounding box.
#     ymax: ymax of bounding box.
#     xmax: xmax of bounding box.
#     color: color to draw bounding box. Default is red.
#                                                                                         thickness: line thickness. Default value is 4.
#     display_str_list: list of strings to display in box
#                       (each to be shown on its own line).
#     use_normalized_coordinates: If True (default), treat coordinates
#       ymin, xmin, ymax, xmax as relative to the image.  Otherwise treat
#       coordinates as absolute.
#   """
#   draw = ImageDraw.Draw(image)
#   im_width, im_height = image.size
#   if use_normalized_coordinates:
#     (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
#                                   ymin * im_height, ymax * im_height)

In [38]:

# def output_to_serial():
#   ser = serial.Serial('COM6', 9600)
#     # Get the lanes we want to open
#     # Maybe we do binary or something?
#     # As in 001001 to open up lanes 3 and 6
#   while True:
#     print("here")
#     ser.write(b"FUCK")
#     print("yo marek")
#     print(ser.read())
#     while ser.read() != "o":
#       print("waiting..")
#       time.sleep(1)
#     print("fuck ya")



In [39]:
from PIL import Image
import math

for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  print(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)

#   boxes = output_dict['detection_boxes']
#   scores = output_dict['detection_scores']
#   width, height = image.size
#   box= np.squeeze(boxes)
#   max_boxes_to_draw=box.shape[0]
#   scores=np.squeeze(scores)
#   min_score_thresh=0.8
    
#   for i in range (min(max_boxes_to_draw, box.shape[0])):
#     if scores [i] > min_score_thresh:
#       position = list("000000")
#       ymin = (int(box[i,0]*height))
#       xmin = (int(box[i,1]*width))
#       ymax = (int(box[i,2]*height))
#       xmax = (int(box[i,3]*width))
#       print(ymin,xmin,ymax,xmax)
#       centre = math.floor(((xmin+xmax)/2)/(width/6)
#       position[centre] = '1'
#       print(centre)
#       print(position)
#       print("".join(position))

  boxes = output_dict['detection_boxes']
  scores = output_dict['detection_scores']
  width, height = image.size
  box= np.squeeze(boxes)
  max_boxes_to_draw=box.shape[0]
  scores=np.squeeze(scores)
  min_score_thresh=0.8
  for i in range (min(max_boxes_to_draw, box.shape[0])):
    if scores [i] > min_score_thresh:
      position = list("000000")
      ymin = (int(box[i,0]*height))
      xmin = (int(box[i,1]*width))
      ymax = (int(box[i,2]*height))
      xmax = (int(box[i,3]*width))
      print(ymin,xmin,ymax,xmax)
      centre = (math.floor(((xmin+xmax)/2)/(width/6))
      position[centre] = '1'
      print(centre)
      print(position)
      print("".join(position))         
    
#       output_to_serial()
    

    
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)

SyntaxError: invalid syntax (<ipython-input-39-29210fad7441>, line 63)